In [ ]:
import pandas as pd
country = "Germany"
year = 2020
gen = "generation_twh"
start = 2010
stop = 2020
years = [year for year in range(start,stop+1)]

In [ ]:
def change_dtype(x):
    try:
        return float(x)
    except:
        return x

In [ ]:
df = pd.read_csv("EER_2022_generation.csv")
emojis = {
    'fossils': 'F',
    'solar': 'S',
    'wind': 'W',
    'nuclear': 'N',
    'hydro': 'H',
    'biomass': 'B',
    'geo': 'G'
}

In [ ]:
df = df.apply(lambda x: change_dtype(x))
# df = df.set_index("fuel_code")

In [ ]:
def filter_by_country(target_country, df_func):
    filt = df_func["country_name"] == target_country
    return filt

def filter_by_year(target_year, df_func):
    filt_year = df_func["year"] == target_year
    return filt_year

filt_country = filter_by_country(country, df)
filt_year = filter_by_year(year, df)

In [ ]:
""" Making full DE dictionary as example """

df_test = df.groupby()


In [ ]:
def create_yearly_gen_dict(df_country, years):
    return {year: df_country[filter_by_year(year, df_country)][gen].to_dict() for year in years}

def create_country_dict(dataframe, country):
    df = dataframe[filter_by_country(country, dataframe)]
    df.set_index("fuel_code",inplace=True)
    return df

In [180]:
""" EXAMPLE WITH FRANCE """

france = create_country_dict(df, 'France')
france_gen = create_yearly_gen_dict(france, years)
france_gen

{2010: {'BIO': 4.44,
  'GAS': 23.76,
  'HARDCOAL': 23.36,
  'HYDRO': 62.71,
  'LIGNITE': 0.0,
  'NUCLEAR': 428.52,
  'OTHFOSS': 10.51,
  'OTHRENEW': 0.48,
  'SOLAR': 0.62,
  'WIND': 9.94},
 2011: {'BIO': 5.04,
  'GAS': 29.49,
  'HARDCOAL': 17.39,
  'HYDRO': 45.74,
  'LIGNITE': 0.0,
  'NUCLEAR': 442.39,
  'OTHFOSS': 12.11,
  'OTHRENEW': 0.54,
  'SOLAR': 2.33,
  'WIND': 12.37},
 2012: {'BIO': 5.3,
  'GAS': 22.75,
  'HARDCOAL': 21.46,
  'HYDRO': 59.83,
  'LIGNITE': 0.0,
  'NUCLEAR': 425.41,
  'OTHFOSS': 12.21,
  'OTHRENEW': 0.51,
  'SOLAR': 4.43,
  'WIND': 15.18},
 2013: {'BIO': 5.68,
  'GAS': 18.39,
  'HARDCOAL': 23.84,
  'HYDRO': 71.92,
  'LIGNITE': 0.0,
  'NUCLEAR': 423.68,
  'OTHFOSS': 11.12,
  'OTHRENEW': 0.5,
  'SOLAR': 5.19,
  'WIND': 16.13},
 2014: {'BIO': 5.99,
  'GAS': 13.16,
  'HARDCOAL': 11.26,
  'HYDRO': 63.77,
  'LIGNITE': 0.0,
  'NUCLEAR': 436.48,
  'OTHFOSS': 11.26,
  'OTHRENEW': 0.56,
  'SOLAR': 6.39,
  'WIND': 17.32},
 2015: {'BIO': 6.53,
  'GAS': 21.14,
  'HARDCOAL': 11

In [ ]:
france_print = {}

In [ ]:
"""
DESIRED DICT FORMAT 


{year 1:
    {'fossils': xx,
    'solar': xx,
    'wind': xx,
    'nuclear': xx,
    'hydro': xx,
    'biomass': xx,
    'others': xx},
year 2:
    {'fossils': xx,
    'solar': xx,
    'wind': xx,
    'nuclear': xx,
    'hydro': xx,
    'biomass': xx,
    'others': xx},
etc.
    
    }
"""

In [ ]:
""" creating dict using for loop"""

#germany = {}
#for year in years:
    #germany[year] = df_test[filter_by_year(year, df_test)][gen].to_dict()
#germany